# 🛳️ EPA_TITANIC_LINSVC_V2 — LinearSVC con GridSearchCV

Este notebook implementa un **Linear Support Vector Classifier (LinearSVC)** para la competencia *Titanic: Machine Learning from Disaster* de Kaggle.  
Se aplica **Feature Engineering**, **preprocesamiento con pipelines** y una **búsqueda de hiperparámetros con GridSearchCV**, evaluando las configuraciones más prometedoras (`C`, `loss`, `class_weight`).  

## 🔹 Flujo del notebook
1. **Carga de datos** desde los archivos oficiales de Kaggle.  
2. **Feature Engineering**: creación de variables como títulos, tamaño familiar, cabina, tarifa por persona, etc.  
3. **Preprocesamiento** con `ColumnTransformer` (imputación, escalado, one-hot encoding).  
4. **GridSearchCV** para encontrar la mejor configuración de LinearSVC.  
5. **Entrenamiento final** y generación de `submission.csv`.  
6. **(Opcional)** Ensamble con `SGDClassifier` y `LogisticRegression` mediante un `VotingClassifier`.  

🎯 **Objetivo:** Optimizar LinearSVC, compararlo con experimentos previos y explorar ensambles de clasificadores lineales.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## 📥 Carga de datos y configuración inicial

En esta celda:
- Fijamos una **semilla reproducible**.
- Detectamos de forma robusta las rutas de `train.csv` y `test.csv` dentro de `/kaggle/input`.
- Cargamos los datos con `pandas`.
- Separamos la variable objetivo (`Survived`) del resto de características.
- Unimos temporalmente `train` y `test` en `full` (útil para aplicar Feature Engineering de forma consistente).
- Mostramos formas y distribución de la etiqueta.



In [4]:
# ======================
# Celda 2 — Load Data
# ======================

# Cargar datasets de Titanic desde el input de Kaggle
train_path = "/kaggle/input/titanic/train.csv"
test_path = "/kaggle/input/titanic/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Variable objetivo
y = train_df["Survived"]

# Features iniciales (dejamos todas salvo Survived para ingeniería posterior)
X = train_df.drop(columns=["Survived"])

print("✅ Datasets cargados correctamente")
print(f"Shape train: {train_df.shape} (con target)")
print(f"Shape test:  {test_df.shape} (sin target)")
print("\nColumnas disponibles en train:")
print(train_df.columns.tolist())


✅ Datasets cargados correctamente
Shape train: (891, 12) (con target)
Shape test:  (418, 11) (sin target)

Columnas disponibles en train:
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


## 🧪 Celda 3 — Feature Engineering (títulos, familia, cabina, tarifas e interacciones)

En esta celda creamos nuevas variables predictivas a partir de las columnas originales:
- **Title** desde `Name` (Mr, Mrs, Miss, Master, Rare).
- **FamilySize** (= `SibSp` + `Parch` + 1) e **IsAlone** (indicador).
- **TicketGroup** (tamaño de grupo por mismo `Ticket`).
- **CabinDeck** (letra del deck extraída desde `Cabin`, con `U` para desconocido).
- **FarePerPerson** (tarifa por persona) y **Age\*Class** (interacción).
  
Aplicamos la misma transformación a `train` y `test` de manera consistente, concatenando primero y separando después.


In [5]:
# ============================
# Celda 3 — Feature Engineering
# ============================

# Unimos train y test para aplicar transformaciones consistentes (no usamos la columna objetivo aquí)
full_df = pd.concat([train_df.drop(columns=["Survived"]), test_df], axis=0, ignore_index=True)  # concatenar sin la y

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica ingeniería de atributos al dataset Titanic.
    No modifica la variable objetivo. Devuelve un nuevo DataFrame con columnas añadidas.
    """
    out = df.copy()  # trabajar sobre una copia para no mutar el original

    # --- Title desde Name ---
    title_series = out["Name"].str.extract(r",\s*([^\.]+)\.")[0].str.strip()  # extraer texto entre coma y punto
    title_map = {  # normalización de títulos infrecuentes
        "Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs",
        "Lady": "Rare", "Countess": "Rare", "Capt": "Rare", "Col": "Rare",
        "Don": "Rare", "Dr": "Rare", "Major": "Rare", "Rev": "Rare",
        "Sir": "Rare", "Jonkheer": "Rare", "Dona": "Rare"
    }
    title_series = title_series.replace(title_map)  # reemplazar alias por categorías consolidadas
    title_series = title_series.where(title_series.isin(["Mr", "Mrs", "Miss", "Master"]), "Rare")  # agrupar el resto en 'Rare'
    out["Title"] = title_series  # asignar nueva columna Title

    # --- Variables de familia ---
    out["FamilySize"] = out["SibSp"].fillna(0) + out["Parch"].fillna(0) + 1  # tamaño de familia (incluye al pasajero)
    out["IsAlone"] = (out["FamilySize"] == 1).astype(int)  # 1 si viaja solo, 0 en caso contrario

    # --- Tamaño de grupo por Ticket ---
    out["TicketGroup"] = out.groupby("Ticket")["Ticket"].transform("count")  # contar cuántos comparten el mismo Ticket

    # --- Cabina/Deck ---
    cabin_str = out["Cabin"].astype(str)  # castear a str para evitar errores con NaN
    deck = cabin_str.str[0]  # tomar la primera letra como deck
    deck = deck.replace("n", "U")  # cuando era NaN → "n", lo mapeamos a 'U' (Unknown)
    out["CabinDeck"] = deck  # asignar deck

    # --- Tarifa por persona ---
    # Evitamos división por cero: si FamilySize==0 (no debería), usamos 1 como denominador
    denom = out["FamilySize"].where(out["FamilySize"] > 0, 1)  # reemplazar 0 por 1 para seguridad
    out["FarePerPerson"] = out["Fare"] / denom  # tarifa individual aproximada

    # --- Interacciones simples ---
    out["Age*Class"] = out["Age"] * out["Pclass"]  # interacción útil edad x clase

    return out  # devolver DataFrame con nuevas columnas

# Aplicar ingeniería de atributos a todo el conjunto unido
full_fe = engineer_features(full_df)  # obtener el DataFrame con features añadidas

# Volver a separar en train y test respetando los tamaños originales
X_fe = full_fe.iloc[:len(train_df)].copy()  # porción correspondiente al train (sin 'Survived')
X_test_fe = full_fe.iloc[len(train_df):].copy()  # porción correspondiente al test

# Adjuntar nuevamente la variable objetivo al train enriquecido para referencia/compatibilidad
X_fe.insert(1, "Survived", train_df["Survived"].values)  # insertar 'Survived' como segunda columna (opcional, para inspección)

# Información de control
print("✅ Feature Engineering aplicado.")
print(f"Train enriquecido: {X_fe.shape} columnas={len(X_fe.columns)}")
print(f"Test enriquecido:  {X_test_fe.shape} columnas={len(X_test_fe.columns)}")

# Vista rápida de columnas nuevas creadas
new_cols = ["Title", "FamilySize", "IsAlone", "TicketGroup", "CabinDeck", "FarePerPerson", "Age*Class"]
print("\nNuevas columnas creadas:", new_cols)

# (Opcional) previsualizar primeras filas para validar
X_fe.head(3)


✅ Feature Engineering aplicado.
Train enriquecido: (891, 19) columnas=19
Test enriquecido:  (418, 18) columnas=18

Nuevas columnas creadas: ['Title', 'FamilySize', 'IsAlone', 'TicketGroup', 'CabinDeck', 'FarePerPerson', 'Age*Class']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone,TicketGroup,CabinDeck,FarePerPerson,Age*Class
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2,0,1,U,3.62500,66.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2,0,2,C,35.64165,38.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1,1,1,U,7.92500,78.0


## 🧼 Celda 4 — Preprocesamiento con `ColumnTransformer` (imputación, escalado y OHE)

En esta celda:
- Definimos **listas de features** numéricas y categóricas (tras la ingeniería de atributos).
- Quitamos columnas no predictivas o redundantes (`Survived`, `Name`, `Ticket`, `Cabin`).
- Preparamos un **pipeline de preprocesamiento**:
  - **Numéricas** → `SimpleImputer(strategy="median")` + `StandardScaler()`
  - **Categóricas** → `SimpleImputer(strategy="most_frequent")` + `OneHotEncoder(handle_unknown="ignore")`
- Dejamos listo `preprocessor` para conectarlo al modelo en la siguiente celda (GridSearchCV con LinearSVC).


In [7]:
# ==============================
# Celda 4 — Preprocesamiento (versión robusta)
# ==============================

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# --- 1) Separar target y descartar columnas no predictivas ---
y = X_fe["Survived"].astype(int)                             # objetivo

drop_cols = ["Survived", "Name", "Ticket", "Cabin"]          # columnas no predictivas o redundantes

# Conservar PassengerId de test para el submission
pid_test = X_test_fe["PassengerId"].astype(int).copy()

# Eliminar solo las columnas que existan en cada DF (evita KeyError)
drop_cols_train = [c for c in drop_cols if c in X_fe.columns]
drop_cols_test  = [c for c in drop_cols if c in X_test_fe.columns]

X_model = X_fe.drop(columns=drop_cols_train)                 # train listo para preprocesar/modelar
X_test_model = X_test_fe.drop(columns=drop_cols_test)        # test listo para preprocesar/modelar

# --- 2) Definir listas de columnas numéricas y categóricas ---
numeric_features = [
    "Age", "SibSp", "Parch", "Fare", "Pclass",
    "FamilySize", "TicketGroup", "FarePerPerson", "Age*Class"
]
categorical_features = ["Sex", "Embarked", "Title", "CabinDeck"]

# Verificación ligera por si hubiera columnas faltantes
missing_in_X = [c for c in numeric_features + categorical_features if c not in X_model.columns]
if missing_in_X:
    print("⚠️ Advertencia: faltan columnas en X_model:", missing_in_X)

# --- 3) Pipelines de preprocesamiento ---
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),           # imputación numérica
    ("scaler", StandardScaler())                             # escalado
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),    # imputación categórica
    ("ohe", OneHotEncoder(handle_unknown="ignore"))          # OHE robusto a niveles nuevos
])

# --- 4) ColumnTransformer que une ambos pipelines ---
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

# --- 5) Información de control ---
print("✅ Preprocesador creado (robusto a columnas ausentes en test).")
print(f"- X_model shape (antes de transformar): {X_model.shape}")
print(f"- X_test_model shape (antes de transformar): {X_test_model.shape}")
print(f"- Numéricas: {numeric_features}")
print(f"- Categóricas: {categorical_features}")
print("ℹ️ 'preprocessor' está listo para conectarse a LinearSVC en GridSearchCV (siguiente celda).")


✅ Preprocesador creado (robusto a columnas ausentes en test).
- X_model shape (antes de transformar): (891, 15)
- X_test_model shape (antes de transformar): (418, 15)
- Numéricas: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize', 'TicketGroup', 'FarePerPerson', 'Age*Class']
- Categóricas: ['Sex', 'Embarked', 'Title', 'CabinDeck']
ℹ️ 'preprocessor' está listo para conectarse a LinearSVC en GridSearchCV (siguiente celda).


## ⚙️ Celda 5 — LinearSVC + GridSearchCV (5-fold estratificado)

Buscamos los mejores hiperparámetros de **LinearSVC**:
- `C`: fuerza de regularización.
- `loss`: `hinge` vs `squared_hinge`.
- `class_weight`: `None` vs `balanced`.
- `dual=False`: recomendado cuando `n_samples > n_features` tras OHE.

Se usa **StratifiedKFold (n_splits=5, shuffle=True)** y `scoring="accuracy"`.


In [8]:
# ==============================
# Celda 5 — GridSearchCV LinearSVC
# ==============================

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd

RANDOM_SEED = 42  # Semilla para reproducibilidad

# 1) Definimos el clasificador base con iteraciones amplias para evitar warnings de convergencia
base_clf = LinearSVC(
    random_state=RANDOM_SEED,  # reproducibilidad
    max_iter=10000             # más iteraciones para facilitar convergencia
)

# 2) Encadenamos preprocesamiento + clasificador en un Pipeline
pipe = Pipeline(steps=[
    ("prep", preprocessor),    # ColumnTransformer definido en la celda 4
    ("clf", base_clf)          # Modelo LinearSVC
])

# 3) Espacio de búsqueda de hiperparámetros
param_grid = {
    "clf__C": [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],     # regularización
    "clf__loss": ["hinge", "squared_hinge"],       # función de pérdida
    "clf__dual": [False],                          # n_samples > n_features ⇒ dual=False
    "clf__class_weight": [None, "balanced"],       # balanceo de clases (opcional)
}

# 4) Definimos la estrategia de CV estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# 5) Configuramos la búsqueda en malla
gs = GridSearchCV(
    estimator=pipe,          # pipeline completo
    param_grid=param_grid,   # grilla de hiperparámetros
    scoring="accuracy",      # métrica de evaluación
    cv=cv,                   # validación cruzada estratificada
    n_jobs=-1,               # usar todos los núcleos disponibles
    verbose=1,               # nivel de verbosidad (muestra el progreso)
    refit=True               # reentrena con los mejores hiperparámetros al final
)

# 6) Ejecutamos la búsqueda de hiperparámetros
gs.fit(X_model, y)

# 7) Reporte principal: mejor score y parámetros
best_mean = gs.best_score_                                   # accuracy promedio CV del mejor set
best_std  = gs.cv_results_["std_test_score"][gs.best_index_] # std del accuracy en CV
print("✅ GridSearchCV finalizado.")
print(f"Mejor CV mean acc: {best_mean:.4f} (std={best_std:.4f})")
print("Mejores parámetros:", gs.best_params_)

# 8) Tabla compacta con los 5 mejores resultados (ordenados por mean_test_score)
results = pd.DataFrame(gs.cv_results_)
cols = ["mean_test_score", "std_test_score", "param_clf__C",
        "param_clf__loss", "param_clf__class_weight"]
top5 = results.sort_values("mean_test_score", ascending=False)[cols].head(5)
print("\nTop-5 combinaciones por accuracy CV:")
display(top5)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
✅ GridSearchCV finalizado.
Mejor CV mean acc: 0.8260 (std=0.0123)
Mejores parámetros: {'clf__C': 0.1, 'clf__class_weight': None, 'clf__dual': False, 'clf__loss': 'squared_hinge'}

Top-5 combinaciones por accuracy CV:


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
                 

,mean_test_score,std_test_score,param_clf__C,param_clf__loss,param_clf__class_weight
1,0.826037,0.012315,0.1,squared_hinge,None
9,0.824901,0.014977,1.0,squared_hinge,None
17,0.824901,0.014977,5.0,squared_hinge,None
21,0.824901,0.014977,10.0,squared_hinge,None
5,0.823784,0.011149,0.5,squared_hinge,None


## 🏁 Celda 6 — Entrenamiento final y `submission.csv`

Entrenamos el **mejor estimador** encontrado por `GridSearchCV` con todos los datos de entrenamiento, predecimos sobre `test` y guardamos el archivo `submission.csv` (columnas: `PassengerId`, `Survived`).


In [9]:
# ==============================
# Celda 6 — Entrenamiento final y submission.csv
# ==============================

# 1) Recuperar el mejor pipeline (preprocesador + LinearSVC tuned)
best_model = gs.best_estimator_

# 2) Entrenar con todo el train
best_model.fit(X_model, y)

# 3) Predecir sobre test
test_pred = best_model.predict(X_test_model)

# 4) Construir y guardar submission
submission = pd.DataFrame({
    "PassengerId": pid_test.values,            # mantener el PassengerId original
    "Survived": test_pred.astype(int)          # cast explícito a int por prolijidad
})

submission_path = "submission.csv"
submission.to_csv(submission_path, index=False)

print(f"✅ Archivo '{submission_path}' creado con {len(submission)} filas y columnas {list(submission.columns)}")
display(submission.head(10))


✅ Archivo 'submission.csv' creado con 418 filas y columnas ['PassengerId', 'Survived']


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


---

# ✅ Conclusiones — EPA_TITANIC_LINSVC_V2

En este notebook implementamos y optimizamos un **Linear Support Vector Classifier (LinearSVC)** para la competencia *Titanic: Machine Learning from Disaster* de Kaggle.

## 🔹 Resultados
- **Mejores hiperparámetros (GridSearchCV):**  
  - `C = 0.1`  
  - `loss = squared_hinge`  
  - `dual = False`  
  - `class_weight = None`  
- **Validación cruzada (5-fold):**  
  - **CV mean acc:** 0.8260  
  - **std:** 0.0123  
- **Kaggle Leaderboard:**  
  - **Score:** 0.77751 → 🥇 *mejor resultado hasta ahora*  

## 🔹 Observaciones
- El ajuste fino de `LinearSVC` logró superar a **SGDClassifier V1 (0.77511)** y a **Logistic Regression (0.77272)**.  
- Los errores en combinaciones inválidas (`hinge` + `dual=False`) se resolvieron limitando la grilla a configuraciones soportadas.  
- El modelo es estable y competitivo considerando la simplicidad de un clasificador lineal.

## 🔹 Próximos pasos
- Explorar **modelos basados en árboles**: `DecisionTree`, `RandomForest`, `GradientBoosting`.  
- Probar **boosting avanzado**: `XGBoost`, `LightGBM`, `CatBoost`.  
- Implementar **VotingClassifier / StackingClassifier** combinando los mejores lineales (LinearSVC + SGD + LogReg).  
- Continuar actualizando el **README** con cada experimento y su score.

---

📌 **Cierre:** LinearSVC V2 se establece como el **mejor modelo lineal en este proyecto Titanic** hasta la fecha, marcando la base para explorar ensambles y modelos no lineales en los siguientes notebooks.
